In [1]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [2]:
x = np.loadtxt('one_boost_test_final.txt', delimiter=',')
x = np.expand_dims(x,axis=2)
train_labels=(x[0:3600,0])-1
print(train_labels)
train_samples=(x[0:3600,1:601])
print(train_samples.shape)

[[0.]
 [0.]
 [0.]
 ...
 [7.]
 [7.]
 [7.]]
(3600, 600, 1)


In [3]:
labels_names = ['Healthy', 'SC DIODE', 'OC DIODE','SC IGBT','OC IGBT', 'OC CAPACITOR', 'SC LOAD','OC LOAD',]

In [4]:
train_labels, train_samples = shuffle(train_labels, train_samples)

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.model_selection import train_test_split

In [6]:
model = keras.Sequential([
 keras.layers.Reshape((200,3), input_shape=(600,1)),
 keras.layers.Conv1D(filters=300, kernel_size=10, 
activation='relu',padding='valid', input_shape=(200, 3)),
 keras.layers.MaxPooling1D(),
 keras.layers.Conv1D(filters=200, kernel_size=5, 
activation='relu',padding='valid'),
 keras.layers.MaxPooling1D(),
 keras.layers.Conv1D(filters=200, kernel_size=3, 
activation='relu',padding='valid'),
 keras.layers.MaxPooling1D(),
 keras.layers.Dropout(0.5),
 keras.layers.Flatten(),
 keras.layers.Dense(200, activation="relu"),
 keras.layers.Dense(100, activation="relu"),
 keras.layers.Dense(8, activation="softmax")])


C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape (Reshape)                    │ (None, 200, 3)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 191, 300)            │           9,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 95, 300)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 91, 200)             │         300,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 45, 200)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 43, 200)             │         120,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 21, 200)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 21, 200)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4200)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 200)                 │         840,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 100)                 │          20,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │             808 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,290,808 (4.92 MB)

 Trainable params: 1,290,808 (4.92 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
data_train, data_test, labels_train, labels_test = train_test_split(train_samples, train_labels, test_size=0.20, random_state=42)

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(train_samples, train_labels, epochs=10)
#model.fit(x=scaled_train_samples, y=train_labels, validation_split = 0.1, batch_size=10, shuffle = True, epochs=30, verbose=2)

Epoch 1/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 12s 67ms/step - accuracy: 0.4893 - loss: 1304.7094
Epoch 2/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.5017 - loss: 2002.9083
Epoch 3/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.7722 - loss: 1897.9600
Epoch 4/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - accuracy: 0.7376 - loss: 25170.5469
Epoch 5/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - accuracy: 0.8583 - loss: 153193.7344
Epoch 6/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9983 - loss: 0.1765
Epoch 7/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - accuracy: 0.9988 - loss: 0.1514
Epoch 8/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - accuracy: 0.9983 - loss: 3.8699
Epoch 9/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.9981 - loss: 0.2515
Epoch 10/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9985 - loss: 0.3635


In [11]:
test_loss, test_acc = model.evaluate(data_test, labels_test)
nofault_loss, nofault_acc = model.evaluate(train_samples[0:449,0:600],train_labels[0:449])
SCD_loss, SCD_acc = model.evaluate(train_samples[450:899,0:600],train_labels[450:899])
OCD_loss, OCD_acc = model.evaluate(train_samples[900:1349,0:600],train_labels[900:1349])
SCI_loss, SCI_acc = model.evaluate(train_samples[1350:1799,0:600],train_labels[1350:1799])
OCI_loss, OCI_acc = model.evaluate(train_samples[1800:2249,0:600],train_labels[1800:2249])
OCC_loss, OCC_acc = model.evaluate(train_samples[2250:2699,0:600],train_labels[2250:2699])
SCL_loss, SCL_acc = model.evaluate(train_samples[2700:3149,0:600],train_labels[2700:3149])
OCL_loss, OCL_acc = model.evaluate(train_samples[3150:3599,0:600],train_labels[3150:3599])

print('\nTest accuracy:', test_acc*100, '%')
prediction=model.predict(data_test)

for i in range(10):
 print("Prediction: ", labels_names[np.argmax(prediction[i])], " Actual: ", labels_names[int(labels_test[i])])

print('\nHealthy accuracy:', nofault_acc*100, '%')
print('\nShort Circuit Diode accuracy:', SCD_acc*100, '%')
print('\nOpen Circuit Diode accuracy:', OCD_acc*100, '%')
print('\nShort Circuit IGBT accuracy:', SCI_acc*100, '%')
print('\nOpen Circuit IGBT accuracy:', OCI_acc*100, '%')
print('\nOpen Circuit Capacitor accuracy:', OCC_acc*100, '%')
print('\nShort Circuit Load accuracy:', SCL_acc*100, '%')
print('\nOpen Circuit Load accuracy:', OCL_acc*100, '%')

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 1.0000 - loss: 0.0000e+00
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 0.0000e+00
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 0.0000e+00
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.0000e+00
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.0000e+00
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.0000e+00
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.0000e+00
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 0.0000e+00
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.0000e+00

Test accuracy: 100.0 %
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
Prediction:  OC CAPACITOR  Actual:  OC CAPACITOR
Prediction:  OC DIODE  Actual:  OC DIODE
Prediction:  Healthy  Actual:  Healthy
Prediction:  OC LOAD  Actual:  OC LOAD
Prediction:  OC LOAD  Actual:  OC LOAD
Prediction:  SC IGBT  Actua

C:\Users\abdul\AppData\Local\Temp\ipykernel_23056\1351805033.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Prediction: ", labels_names[np.argmax(prediction[i])], " Actual: ", labels_names[int(labels_test[i])])


In [17]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(confusion_matrix(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
[[ 86   0   0   0   0   0   0   0]
 [  0 100   0   0   0   0   0   0]
 [  0   0  93   0   0   0   0   0]
 [  0   0   0  97   0   0   0   0]
 [  0   0   0   0  81   0   0   0]
 [  0   0   0   0   0  87   0   0]
 [  0   0   0   0   0   0  84   0]
 [  0   0   0   0   0   0   0  92]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        86
           1       1.00      1.00      1.00       100
           2       1.00      1.00      1.00        93
           3       1.00      1.00      1.00        97
           4       1.00      1.00      1.00        81
           5       1.00      1.00      1.00        87
           6       1.00      1.00      1.00        84
           7       1.00      1.00      1.00        92

    accuracy                           1.00       720
   macro avg       1.00      1.00      1.00       720
weighted avg       1.00      1.00      1.00       720



In [12]:
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load and preprocess data
x = np.loadtxt('one_boost_test_final.txt', delimiter=',')
x = np.expand_dims(x, axis=2)  # (3600, 600, 1)
train_labels = (x[0:3600, 0]) - 1
train_samples = x[0:3600, 1:601]
train_labels, train_samples = shuffle(train_labels, train_samples)

# Split data
X_train, X_test, y_train, y_test = train_test_split(train_samples, train_labels, test_size=0.2)

# Convert labels to integer and one-hot encode if using categorical
y_train = y_train.astype('int')
y_test = y_test.astype('int')


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

lstm_model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(600, 1)),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(100, activation='relu'),
    Dense(8, activation='softmax')  # 8 classes
])

lstm_model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

lstm_model.summary()


C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 600, 128)            │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 600, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │           6,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8)                   │             808 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 123,276 (481.55 KB)

 Trainable params: 123,276 (481.55 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 68s 720ms/step - accuracy: 0.3991 - loss: 1.6044 - val_accuracy: 0.5174 - val_loss: 1.1608
Epoch 2/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 58s 713ms/step - accuracy: 0.5246 - loss: 1.1474 - val_accuracy: 0.6285 - val_loss: 0.6800
Epoch 3/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 57s 706ms/step - accuracy: 0.5445 - loss: 1.0601 - val_accuracy: 0.6076 - val_loss: 0.8316
Epoch 4/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 58s 718ms/step - accuracy: 0.5754 - loss: 0.9515 - val_accuracy: 0.5104 - val_loss: 0.8203
Epoch 5/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 89s 800ms/step - accuracy: 0.6534 - loss: 0.7190 - val_accuracy: 0.7778 - val_loss: 0.4324
Epoch 6/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 49s 599ms/step - accuracy: 0.8520 - loss: 0.4038 - val_accuracy: 0.4931 - val_loss: 1.0360
Epoch 7/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 50s 613ms/step - accuracy: 0.7667 - loss: 0.5781 - val_accuracy: 1.0000 - val_loss: 0.1251
Epoch 8/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 53s 654ms/step - accuracy: 0.8609 - loss: 0.3850 - val_accu

In [15]:
test_loss, test_acc = lstm_model.evaluate(X_test, y_test)
print("LSTM Test Accuracy:", test_acc * 100, "%")


23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 284ms/step - accuracy: 1.0000 - loss: 0.1204
LSTM Test Accuracy: 100.0 %


In [16]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(confusion_matrix(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))


23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 287ms/step
[[ 86   0   0   0   0   0   0   0]
 [  0 100   0   0   0   0   0   0]
 [  0   0  93   0   0   0   0   0]
 [  0   0   0  97   0   0   0   0]
 [  0   0   0   0  81   0   0   0]
 [  0   0   0   0   0  87   0   0]
 [  0   0   0   0   0   0  84   0]
 [  0   0   0   0   0   0   0  92]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        86
           1       1.00      1.00      1.00       100
           2       1.00      1.00      1.00        93
           3       1.00      1.00      1.00        97
           4       1.00      1.00      1.00        81
           5       1.00      1.00      1.00        87
           6       1.00      1.00      1.00        84
           7       1.00      1.00      1.00        92

    accuracy                           1.00       720
   macro avg       1.00      1.00      1.00       720
weighted avg       1.00      1.00      1.00       720

